In [1]:
# import packages
import os
import sys
import numpy as np
import tensorflow as tf
tf.__version__

'1.12.0'

## tf.constant()

In [2]:
# constant is used to hold fixed numeric values
node1 = tf.constant(3.0, tf.float32) # nodes are nothing but tensors
node2 = tf.constant(4.5) # implicitly applied: float32

In [3]:
node1, node2

(<tf.Tensor 'Const:0' shape=() dtype=float32>,
 <tf.Tensor 'Const_1:0' shape=() dtype=float32>)

In [4]:
# create a session object to evaluate nodes
with tf.Session() as sess:
    sess.run(node1)

In [5]:
# you can evaluate multiple nodes at once
with tf.Session() as sess:
    print(sess.run([node1, node2]))

[3.0, 4.5]


In [6]:
node3 = tf.add(node1, node2) # add two nodes and assign to another node

In [7]:
node3

<tf.Tensor 'Add:0' shape=() dtype=float32>

In [8]:
# run the computation graph
with tf.Session() as sess:
    print(sess.run(node3))

7.5


## tf.placeholders()

In [9]:
# placeholders are used to hold variable input
a = tf.placeholder(tf.int32) # define the data type of the input
b = tf.placeholder(tf.int32)

In [10]:
adder_node = tf.add(a, b) # add two nodes

In [11]:
# evaluate computation graph
with tf.Session() as sess:
    print(sess.run(adder_node, feed_dict={a: 2, b: 4})) # use feed_dict to pass input to nodes

6


In [12]:
# evaluate computation graph
with tf.Session() as sess:
    print(sess.run(adder_node, feed_dict={a:[1, 2], b:[3, 10]})) # pass multiple input to nodes

[ 4 12]


In [13]:
with tf.Session() as sess:
    result = sess.run(adder_node, feed_dict={a:[2, 3, 4], b:[4, 5, 6]}) # save the result into another node
    print(result)

[ 6  8 10]


In [14]:
result, result.shape

(array([ 6,  8, 10], dtype=int32), (3,))

## tf.Variable()

In [15]:
# variable is used to hold values that keep changing during the process
# variable maintains state in the graph across calls to run
variable_a = tf.Variable(1.0, tf.float32)
variable_a

<tf.Variable 'Variable:0' shape=() dtype=float32_ref>

In [16]:
variable_b = tf.Variable([1.0, 2.0], tf.float32)
variable_b

<tf.Variable 'Variable_1:0' shape=(2,) dtype=float32_ref>

## Forward Pass Over Linear Regression with Unit Feature

In [17]:
weight = tf.Variable([1.0], tf.float32) # set intial weight as 1.0
bias = tf.Variable([1.0], tf.float32) # set initial weight as 1.0
x = tf.placeholder(tf.float32) # unit feature

model = weight * x + bias # vectorize linear model

# create placeholder for predicted value
y_original = tf.placeholder(tf.float32)

# create a error metric
error_measure = tf.squared_difference(model, y_original)

# create a loss function
loss_function = tf.reduce_mean(error_measure)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # now run to evaluate the loss function
    print("Loss = ", sess.run(loss_function, feed_dict={x:[1, 2, 3, 4], y_original:[-1, -2, -3, -4]}))
    # predict
    print(sess.run(model, feed_dict={x:[1, 2, 3, 4], y_original:[-1, -2, -3, -4]}))

Loss =  41.0
[2. 3. 4. 5.]


In [18]:
# now lets change weight and bais
weight = tf.Variable([-1.0], tf.float32) # set intial weight as 1.0
bias = tf.Variable([0.0], tf.float32) # set initial weight as 1.0
x = tf.placeholder(tf.float32) # unit feature

model = weight * x + bias # vectorize linear model

# create placeholder for predicted value
y_original = tf.placeholder(tf.float32)

# create a error metric
error_measure = tf.squared_difference(model, y_original)

# create a loss function
loss_function = tf.reduce_mean(error_measure)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # now run to evaluate the loss function
    print("Loss = ", sess.run(loss_function, feed_dict={x:[1, 2, 3, 4], y_original:[-1, -2, -3, -4]}))
    # predict
    print(sess.run(model, feed_dict={x:[1, 2, 3, 4], y_original:[-1, -2, -3, -4]}))

Loss =  0.0
[-1. -2. -3. -4.]


## Linear Regression Using Gradient Descent Optimizer

In [19]:
# get dummy data for now
# training data
x_train = [1, 2, 3, 4, 5, 6, 7, 8, 9]
y_train = [2, 4, 6, 8, 10, 12, 14, 16, 18]

# validation data
x_val = [0, 10, 20, 30, 40, 50]
y_val = [0, 20, 40, 60, 80, 100]

# test data
x_test = [0, -1, 100]
y_test = [0, -2, 200]

In [20]:
# first set some initial values for weight and bias: same as before
weight = tf.Variable([0.0])
bias = tf.Variable([0.0])

x = tf.placeholder(tf.float32) # placeholder
y = tf.placeholder(tf.float32)

linear_model = tf.add(tf.multiply(weight, x), bias) # model

loss_function = tf.reduce_mean(tf.square(tf.subtract(linear_model, y))) # minimize the loss function = mean squared difference

alpha = tf.constant(0.01) # set fixed learning rate

optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha) # use an optimizer to optimize the weights and bias

# use the optimizer to minimise the loss function
trainer = optimizer.minimize(loss_function)

In [21]:
# run the mode
with tf.Session() as sess:
    print("Training...")
    sess.run(tf.global_variables_initializer()) # initialize variables
    # run the optimizer for 1000 iterations
    for i in range(1000):
        sess.run(trainer, feed_dict={x:x_train, y:y_train})
        # print("Train Loss = ", sess.run(loss_function, feed_dict={x:x_train, y:y_train}))
        if i % 100 == 0:
            print("\t\tValidation Loss = ", sess.run(loss_function, feed_dict={x:x_val, y:y_val}))
            print("\t\tWeights = ", sess.run(weight))
            print("\t\tBias = ", sess.run(bias))
    # now perform prediction
    print("Testing...")
    print("Test Loss = ", sess.run(loss_function, feed_dict={x:x_test, y:y_test}))
    print("Prediction = ", sess.run(linear_model, feed_dict={x:x_test})) # expected values are in y_test

Training...
		Validation Loss =  485.66968
		Weights =  [1.2666667]
		Bias =  [0.19999999]
		Validation Loss =  0.6782417
		Weights =  [1.9674879]
		Bias =  [0.2045743]
		Validation Loss =  0.2977697
		Weights =  [1.9784577]
		Bias =  [0.13554992]
		Validation Loss =  0.13073128
		Weights =  [1.9857261]
		Bias =  [0.08981472]
		Validation Loss =  0.057394724
		Weights =  [1.9905423]
		Bias =  [0.05951079]
		Validation Loss =  0.025199456
		Weights =  [1.9937333]
		Bias =  [0.03943158]
		Validation Loss =  0.011062431
		Weights =  [1.9958477]
		Bias =  [0.0261272]
		Validation Loss =  0.0048566204
		Weights =  [1.9972488]
		Bias =  [0.0173117]
		Validation Loss =  0.002132545
		Weights =  [1.998177]
		Bias =  [0.0114706]
		Validation Loss =  0.0009361114
		Weights =  [1.9987922]
		Bias =  [0.0076003]
Testing...
Test Loss =  0.0019101078
Prediction =  [ 5.0567007e-03 -1.9941397e+00  1.9992470e+02]


As we can see that we didnt get the exact value but its very close and error has been reduced drastically.